In [7]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [8]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

In [9]:
#imports
import os
import time
import pandas as pd
import numpy as np
import sys
from io import StringIO
import glob
if not hasattr(sys, "version_info") or sys.version_info < (3, 5):
  raise SystemExit("This program requires Python 3.5 or later.")
import csv
import matplotlib
#matplotlib.use('nbagg')
import matplotlib.pyplot as plt
## https://github.com/ipython/ipyparallel
import ipyparallel as ipp
from IPython.display import JSON
import json
import statistics as st
from pympler import asizeof
from hurry.filesize import size
import tabletext
import sqlite3
import shutil

In [10]:
def mstime():
    import time
    return round(time.time() * 1000)

def sync():
    import os
    os.system('sync')
    
def touch(path):
    import os
    with open(path, 'a'):
        os.utime(path, None)
    sync()
    
def unic(l: list) -> list :
    return list(set(l))

def cleanupTmp():
    print("clean tmp in %s" % cutPathes(tmpFolder))
    if tmpFolder == "/tmp":
        print("invalid tmp folder!")
        exit(-1)
    for filename in os.listdir(tmpFolder):
        file_path = os.path.join(tmpFolder, filename)
        # print("remove %s" % file_path)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (cutPathes(file_path), e))

In [11]:
# read env
envHome = os.environ.get('HOME')
envRamHome = os.environ.get('RAMCASHE')

#init env
tmpFolder = "%s/tmp" % envRamHome
if os.path.isdir(tmpFolder):
    shutil.rmtree(tmpFolder, ignore_errors=True)

os.mkdir(tmpFolder)

In [12]:
data_sources = glob.glob("data/*_shops.xml")
plt.rcParams['figure.figsize'] = [30/2.54, 20/2.54]

# tags
client = ipp.Client()
workers = client[:]

midDb = "%s/midData/mid.db" % os.getcwd()
midDbExample = "%s/midData/mid.example.db" % os.getcwd()
midDbTmp = "%s/mid.%d.db" % (envRamHome, mstime())
baseDir = os.getcwd() + "/"

def cutPathes(s: str) -> str:
    if s.startswith(baseDir):
        return s[len(baseDir)::]
    if s.startswith(envRamHome):
        return s[len(envRamHome)::]
    return s

print("engines count: %d" % len(client.ids))
print("source files count: %d" % len(data_sources))
print(tmpFolder)

engines count: 4
source files count: 2120
/home/ed/ramcashe/tmp


In [13]:
workers["tmpFolder"] = tmpFolder
workers["dbquery_cratetable"] = '''
CREATE TABLE "ttids" (
	"rowid"	INTEGER NOT NULL UNIQUE,
	"shapshotid"	TEXT NOT NULL,
	"ttid"	INTEGER NOT NULL,
	"buy_list"	TEXT NOT NULL,
	"sell_list"	TEXT NOT NULL,
	"capacity"	INTEGER NOT NULL,
	"buystat_avg"	REAL NOT NULL,
	"buystat_min"	REAL NOT NULL,
	"buystat_max"	REAL NOT NULL,
	"buystat_median"	REAL NOT NULL,
	"buystat_gap"	REAL NOT NULL,
	"buystat_stdev"	REAL NOT NULL,
	"sellstat_avg"	REAL NOT NULL,
	"sellstat_min"	REAL NOT NULL,
	"sellstat_max"	REAL NOT NULL,
	"sellstat_median"	REAL NOT NULL,
	"sellstat_gap"	REAL NOT NULL,
	"sellstat_stdev"	REAL NOT NULL,
	PRIMARY KEY("rowid" AUTOINCREMENT)
);'''
dbquery_cratetable = workers["dbquery_cratetable"][0]

workers["dbquery_insert"] = '''
INSERT INTO 'ttids'(
"shapshotid",
"ttid",
"buy_list",
"sell_list",
"capacity",
"buystat_avg",
"buystat_min",
"buystat_max",
"buystat_median",
"buystat_gap",
"buystat_stdev",
"sellstat_avg",
"sellstat_min",
"sellstat_max",
"sellstat_median",
"sellstat_gap",
"sellstat_stdev"
) VALUES (
?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);'''
dbquery_insert = workers["dbquery_insert"][0]

In [14]:
def processSnapshot(fname):
    import xml.etree.ElementTree as ET
    import statistics as st
    import sqlite3
    import os
    import time
    from pathlib import Path

    def mstime():
        return round(time.time() * 1000)
    def sync():
        os.system('sync')
    def touch(path):
        with open(path, 'a'):
            os.utime(path, None)
        sync()
    
    log = []
    p = Path(fname)
    baseDir = str(p.parent)
    dbName = "%s/%s.%s.db" % (tmpFolder, str(p.name), mstime())
    log.append("process %s. base: %s, tmp database: %s" % (fname, baseDir, dbName))
    def processShop(tree, time):
        shopData = []
        ttidList = []
        for good in tree:
            goodData = {}
            goodData["ttid"] = int(good.findall("ttid")[0].text)
            ttidList.append(goodData["ttid"])
            goodData["time"] = time
            goodData["shop"] = tree.attrib["name"]
            goodData["shopid"] = tree.attrib["id"]
            goodData["shopmoney"] = tree.attrib["money"]
            sellElems = good.findall("sell")
            if sellElems:
                goodData["sell"] = float(sellElems[0].text)
            else:
                goodData["sell"] = 0.0

            buyElems = good.findall("buy")
            if buyElems:
                goodData["buy"] = float(buyElems[0].text)
            else:
                goodData["buy"] = 0.0
            goodData["count"] = int(good.findall("count")[0].text)
            goodData["thingname"] = good.findall("thingname")[0].text
            shopData.append(goodData)
        return (shopData, ttidList)
    

    if os.path.exists(dbName):
        os.remove(dbName)
    sync()
    touch(dbName)
    timetag = "lastupdatetime"
    shopTag = "shop"
    xmltree = ET.parse(fname)
    xmlroot = xmltree.getroot()
    timeelem = xmlroot.find(timetag)
    dataTime = int(timeelem.text)
    orders = []
    ttids = []
    con = sqlite3.connect(dbName)    
    cur = con.cursor()
    q = dbquery_cratetable.replace("\n", " ")
    cur.execute(q)
    for shop in xmlroot.findall(shopTag):
        rc = processShop(shop, dataTime)
        shopData = rc[0]
        ttidList = rc[1]
        orders.extend(shopData)
        ttids.extend(ttidList)
    ttid_summary = []
    for ttid in ttids:
        ttid_stat = {}
        ttid_stat["ttid"] = ttid
        ttid_stat["time"] = dataTime
        ttid_stat["buy"] = []
        ttid_stat["sell"] = []
        ttid_stat["capacity"] = 0
        for order in orders:
            if order["ttid"] == ttid:
                ttid_stat["capacity"] += order["count"]
                if order["buy"] > 0:
                    ttid_stat["buy"].append(order["buy"] )
                if order["sell"] > 0:
                    ttid_stat["sell"].append(order["sell"] )  
        
        ttid_stat["buystat"] = {}
        if ttid_stat["buy"]:
            ttid_stat["buystat"]["avg"] = st.mean(ttid_stat["buy"])
            ttid_stat["buystat"]["min"] = min(ttid_stat["buy"])
            ttid_stat["buystat"]["max"] = max(ttid_stat["buy"])
            ttid_stat["buystat"]["gap"] = ttid_stat["buystat"]["max"] - ttid_stat["buystat"]["min"]
            ttid_stat["buystat"]["median"] = st.median(ttid_stat["buy"])
            ttid_stat["buystat"]["stdev"] = 0
            if len(ttid_stat["buy"]) >= 2:
                ttid_stat["buystat"]["stdev"] = st.stdev(ttid_stat["buy"], ttid_stat["buystat"]["avg"])
        else:
            ttid_stat["buystat"]["avg"] = 0.0
            ttid_stat["buystat"]["min"] = 0.0
            ttid_stat["buystat"]["max"] = 0.0
            ttid_stat["buystat"]["gap"] = 0.0
            ttid_stat["buystat"]["median"] = 0.0
            ttid_stat["buystat"]["stdev"] = 0.0
            
        ttid_stat["sellstat"] = {}
        if len(ttid_stat["sell"]) >= 2:
            ttid_stat["sellstat"]["avg"] = st.mean(ttid_stat["sell"])
            ttid_stat["sellstat"]["min"] = min(ttid_stat["sell"])
            ttid_stat["sellstat"]["max"] = max(ttid_stat["sell"])
            ttid_stat["sellstat"]["gap"] = ttid_stat["sellstat"]["max"] - ttid_stat["sellstat"]["min"]
            ttid_stat["sellstat"]["median"] = st.median(ttid_stat["sell"])
            ttid_stat["sellstat"]["stdev"] = 0
            if len(ttid_stat["sell"]) >= 2:
                ttid_stat["sellstat"]["stdev"] = st.stdev(ttid_stat["sell"], ttid_stat["sellstat"]["avg"])
        else:
            ttid_stat["sellstat"]["avg"] = 0.0
            ttid_stat["sellstat"]["min"] = 0.0
            ttid_stat["sellstat"]["max"] = 0.0
            ttid_stat["sellstat"]["median"] = 0.0
            ttid_stat["sellstat"]["gap"] = 0.0
            ttid_stat["sellstat"]["stdev"] = 0.0
        
        ttid_summary.append(ttid_stat)
        q = dbquery_insert.replace("\n", " ")
        try:
            cur.execute(q, (ttid_stat["time"],
                                            ttid_stat["ttid"],
                                            str(ttid_stat["buy"]),
                                            str(ttid_stat["sell"]),
                                            ttid_stat["capacity"],
                                            ttid_stat["buystat"]["avg"],
                                            ttid_stat["buystat"]["min"],
                                            ttid_stat["buystat"]["max"],
                                            ttid_stat["buystat"]["median"],
                                            ttid_stat["buystat"]["gap"],
                                            ttid_stat["buystat"]["stdev"],
                                            ttid_stat["sellstat"]["avg"],
                                            ttid_stat["sellstat"]["min"],
                                            ttid_stat["sellstat"]["max"],
                                            ttid_stat["sellstat"]["median"],
                                            ttid_stat["sellstat"]["gap"],
                                            ttid_stat["sellstat"]["stdev"]))
        except Exception as e:
            log.append("db exception %s in query %s" % (q, str(e)))
            return (None, fname, log)
    
    con.commit()
    con.close()
    sync()
    return (dbName, fname, log)

In [15]:
%%time
cleanupTmp()
sync()
full_pathes = [("%s/%s" % (os.getcwd(), f)) for f in data_sources]

def syncExec(pathes):
    parts = []
    counter = 0
    for p in pathes:
        print("%s - try (%d/%d)" % (p, counter, len(full_pathes)))
        parts.append(processSnapshot(p))
        print("%s - done (%d/%d)" % (p, counter, len(full_pathes)))
        sys.stdout.flush()
        counter += 1
    return parts
parts = workers.map_async(processSnapshot, full_pathes).get()
#parts = syncExec(full_pathes)
print("data size %s" % size(asizeof.asizeof(parts)))


clean tmp in /tmp
data size 1M
CPU times: user 179 ms, sys: 69 ms, total: 248 ms
Wall time: 28min 57s


In [22]:
#%%time
import shutil
def mprint(msg: str):
    print(msg)
    sys.stdout.flush()
   

outCon = sqlite3.connect(midDbTmp)
outcur = outCon.cursor()

backupDb = "%s/midData/mid_BU%d.db" % (os.getcwd(), mstime())
if os.path.isfile(midDbTmp):
    shutil.copyfile(midDbTmp, backupDb)
    sync()
mprint("midDbExample %s " % cutPathes(midDbExample))
mprint("midDb %s " % cutPathes(midDbTmp))
shutil.copyfile(midDbExample, midDbTmp)  
sync()

totalMerged = 0
partsCount = len(parts)
partsIndex = 0
for part in parts:
    partstart = time.time()
    if not part[0]:
        print("cannot process %s" % part[1])
        for logline in part[2]:
            print(logline)
        break
    dbName = part[0]
    mprint("process %s. (part %d/%d)" % (cutPathes(dbName), partsIndex, partsCount))
    secondDbName = 'db' + str(partsCount) + str(mstime())
    outcur.execute("ATTACH DATABASE ? as ? ;", (dbName, secondDbName))
    column_names="shapshotid,ttid,buy_list,sell_list,capacity,buystat_avg,buystat_min,buystat_max,buystat_median,buystat_gap,buystat_stdev,sellstat_avg,sellstat_min,sellstat_max,sellstat_median,sellstat_gap,sellstat_stdev"
    q = "INSERT INTO ttids (%s) SELECT %s FROM %s.ttids;" % (column_names, column_names, secondDbName)
    outcur.execute(q)
    outCon.commit()
    outcur.execute("DETACH  DATABASE %s ;" % secondDbName)
    partdone = time.time()
    partelapsed = partdone - partstart
    mprint("done -- %d parts merged (of %d). total %d. time %s " % (partsIndex, partsCount, totalMerged, str(partelapsed)))
    partsIndex += 1
    
outCon.commit()
q = "SELECT COUNT(ttid) FROM ttids"
outcur.execute(q)
print("done. records in mid db: %d" % outcur.fetchall()[0][0])
outCon.close()
shutil.copyfile(midDbTmp, midDb)

midDbExample midData/mid.example.db 
midDb /mid.1633023094121.db 
process /tmp/1628517800_shops.xml.1633023097735.db. (part 0/2120)
done -- 0 parts merged (of 2120). total 0. time 0.009441375732421875 
process /tmp/1628519402_shops.xml.1633023101079.db. (part 1/2120)
done -- 1 parts merged (of 2120). total 0. time 0.009500503540039062 
process /tmp/1628521201_shops.xml.1633023105544.db. (part 2/2120)
done -- 2 parts merged (of 2120). total 0. time 0.011394500732421875 
process /tmp/1628523001_shops.xml.1633023109282.db. (part 3/2120)
done -- 3 parts merged (of 2120). total 0. time 0.00997471809387207 
process /tmp/1628526601_shops.xml.1633023111768.db. (part 4/2120)
done -- 4 parts merged (of 2120). total 0. time 0.009067773818969727 
process /tmp/1628528401_shops.xml.1633023114339.db. (part 5/2120)
done -- 5 parts merged (of 2120). total 0. time 0.01153421401977539 
process /tmp/1628530201_shops.xml.1633023117027.db. (part 6/2120)
done -- 6 parts merged (of 2120). total 0. time 0.0092

done -- 59 parts merged (of 2120). total 0. time 0.009911060333251953 
process /tmp/1628629201_shops.xml.1633023227639.db. (part 60/2120)
done -- 60 parts merged (of 2120). total 0. time 0.008313655853271484 
process /tmp/1628631001_shops.xml.1633023229654.db. (part 61/2120)
done -- 61 parts merged (of 2120). total 0. time 0.007696628570556641 
process /tmp/1628632801_shops.xml.1633023231550.db. (part 62/2120)
done -- 62 parts merged (of 2120). total 0. time 0.007068157196044922 
process /tmp/1628634602_shops.xml.1633023233186.db. (part 63/2120)
done -- 63 parts merged (of 2120). total 0. time 0.009051322937011719 
process /tmp/1628636401_shops.xml.1633023234613.db. (part 64/2120)
done -- 64 parts merged (of 2120). total 0. time 0.007159709930419922 
process /tmp/1628638201_shops.xml.1633023235808.db. (part 65/2120)
done -- 65 parts merged (of 2120). total 0. time 0.013979434967041016 
process /tmp/1628640001_shops.xml.1633023236963.db. (part 66/2120)
done -- 66 parts merged (of 2120).

process /tmp/1628735401_shops.xml.1633023337970.db. (part 119/2120)
done -- 119 parts merged (of 2120). total 0. time 0.005117654800415039 
process /tmp/1628737201_shops.xml.1633023338636.db. (part 120/2120)
done -- 120 parts merged (of 2120). total 0. time 0.007670879364013672 
process /tmp/1628739001_shops.xml.1633023339292.db. (part 121/2120)
done -- 121 parts merged (of 2120). total 0. time 0.006120204925537109 
process /tmp/1628742601_shops.xml.1633023340013.db. (part 122/2120)
done -- 122 parts merged (of 2120). total 0. time 0.006453514099121094 
process /tmp/1628744401_shops.xml.1633023341212.db. (part 123/2120)
done -- 123 parts merged (of 2120). total 0. time 0.0063626766204833984 
process /tmp/1628746201_shops.xml.1633023342402.db. (part 124/2120)
done -- 124 parts merged (of 2120). total 0. time 0.009085893630981445 
process /tmp/1628748001_shops.xml.1633023344568.db. (part 125/2120)
done -- 125 parts merged (of 2120). total 0. time 0.008728504180908203 
process /tmp/162874

process /tmp/1628847001_shops.xml.1633023453280.db. (part 178/2120)
done -- 178 parts merged (of 2120). total 0. time 0.015996932983398438 
process /tmp/1628848801_shops.xml.1633023456037.db. (part 179/2120)
done -- 179 parts merged (of 2120). total 0. time 0.016645193099975586 
process /tmp/1628850601_shops.xml.1633023458517.db. (part 180/2120)
done -- 180 parts merged (of 2120). total 0. time 0.02498650550842285 
process /tmp/1628852401_shops.xml.1633023460908.db. (part 181/2120)
done -- 181 parts merged (of 2120). total 0. time 0.01053619384765625 
process /tmp/1628854201_shops.xml.1633023463183.db. (part 182/2120)
done -- 182 parts merged (of 2120). total 0. time 0.007904767990112305 
process /tmp/1628856001_shops.xml.1633023465405.db. (part 183/2120)
done -- 183 parts merged (of 2120). total 0. time 0.016547441482543945 
process /tmp/1628857801_shops.xml.1633023468003.db. (part 184/2120)
done -- 184 parts merged (of 2120). total 0. time 0.009905576705932617 
process /tmp/162885960

process /tmp/1628955001_shops.xml.1633023603196.db. (part 237/2120)
done -- 237 parts merged (of 2120). total 0. time 0.011874675750732422 
process /tmp/1628958601_shops.xml.1633023606654.db. (part 238/2120)
done -- 238 parts merged (of 2120). total 0. time 0.010024070739746094 
process /tmp/1628960401_shops.xml.1633023609483.db. (part 239/2120)
done -- 239 parts merged (of 2120). total 0. time 0.009864330291748047 
process /tmp/1628962201_shops.xml.1633023612388.db. (part 240/2120)
done -- 240 parts merged (of 2120). total 0. time 0.0118408203125 
process /tmp/1628964001_shops.xml.1633023615357.db. (part 241/2120)
done -- 241 parts merged (of 2120). total 0. time 0.021758079528808594 
process /tmp/1628965801_shops.xml.1633023618431.db. (part 242/2120)
done -- 242 parts merged (of 2120). total 0. time 0.010401010513305664 
process /tmp/1628967601_shops.xml.1633023621672.db. (part 243/2120)
done -- 243 parts merged (of 2120). total 0. time 0.010082721710205078 
process /tmp/1628969401_s

...

done -- 1931 parts merged (of 2120). total 0. time 0.011779308319091797 
process /tmp/1632261601_shops.xml.1633024240957.db. (part 1932/2120)
done -- 1932 parts merged (of 2120). total 0. time 0.00963139533996582 
process /tmp/1632263402_shops.xml.1633024244408.db. (part 1933/2120)
done -- 1933 parts merged (of 2120). total 0. time 0.011039018630981445 
process /tmp/1632265201_shops.xml.1633024247921.db. (part 1934/2120)
done -- 1934 parts merged (of 2120). total 0. time 0.009748458862304688 
process /tmp/1632267001_shops.xml.1633024251625.db. (part 1935/2120)
done -- 1935 parts merged (of 2120). total 0. time 0.008223295211791992 
process /tmp/1632268801_shops.xml.1633024254479.db. (part 1936/2120)
done -- 1936 parts merged (of 2120). total 0. time 0.008618354797363281 
process /tmp/1632270601_shops.xml.1633024256932.db. (part 1937/2120)
done -- 1937 parts merged (of 2120). total 0. time 0.006121397018432617 
process /tmp/1632272401_shops.xml.1633024258218.db. (part 1938/2120)
done --

done -- 1989 parts merged (of 2120). total 0. time 0.00773167610168457 
process /tmp/1632367801_shops.xml.1633024437632.db. (part 1990/2120)
done -- 1990 parts merged (of 2120). total 0. time 0.007623434066772461 
process /tmp/1632369601_shops.xml.1633024439410.db. (part 1991/2120)
done -- 1991 parts merged (of 2120). total 0. time 0.007453441619873047 
process /tmp/1632371402_shops.xml.1633024441381.db. (part 1992/2120)
done -- 1992 parts merged (of 2120). total 0. time 0.010536670684814453 
process /tmp/1632373201_shops.xml.1633024443393.db. (part 1993/2120)
done -- 1993 parts merged (of 2120). total 0. time 0.008836746215820312 
process /tmp/1632375002_shops.xml.1633024445960.db. (part 1994/2120)
done -- 1994 parts merged (of 2120). total 0. time 0.0081634521484375 
process /tmp/1632376801_shops.xml.1633024448570.db. (part 1995/2120)
done -- 1995 parts merged (of 2120). total 0. time 0.010209321975708008 
process /tmp/1632378601_shops.xml.1633024452606.db. (part 1996/2120)
done -- 1

done -- 2047 parts merged (of 2120). total 0. time 0.012813091278076172 
process /tmp/1632474001_shops.xml.1633024614907.db. (part 2048/2120)
done -- 2048 parts merged (of 2120). total 0. time 0.01334834098815918 
process /tmp/1632475801_shops.xml.1633024618461.db. (part 2049/2120)
done -- 2049 parts merged (of 2120). total 0. time 0.012950420379638672 
process /tmp/1632477602_shops.xml.1633024622787.db. (part 2050/2120)
done -- 2050 parts merged (of 2120). total 0. time 0.01285243034362793 
process /tmp/1632479401_shops.xml.1633024626673.db. (part 2051/2120)
done -- 2051 parts merged (of 2120). total 0. time 0.012989282608032227 
process /tmp/1632481201_shops.xml.1633024630128.db. (part 2052/2120)
done -- 2052 parts merged (of 2120). total 0. time 0.020563840866088867 
process /tmp/1632483001_shops.xml.1633024633970.db. (part 2053/2120)
done -- 2053 parts merged (of 2120). total 0. time 0.010542154312133789 
process /tmp/1632484801_shops.xml.1633024637198.db. (part 2054/2120)
done -- 

done -- 2105 parts merged (of 2120). total 0. time 0.020511627197265625 
process /tmp/1632587402_shops.xml.1633024785699.db. (part 2106/2120)
done -- 2106 parts merged (of 2120). total 0. time 0.026762723922729492 
process /tmp/1632589201_shops.xml.1633024788949.db. (part 2107/2120)
done -- 2107 parts merged (of 2120). total 0. time 0.015491247177124023 
process /tmp/1632591001_shops.xml.1633024791490.db. (part 2108/2120)
done -- 2108 parts merged (of 2120). total 0. time 0.011883974075317383 
process /tmp/1632592801_shops.xml.1633024794146.db. (part 2109/2120)
done -- 2109 parts merged (of 2120). total 0. time 0.021622180938720703 
process /tmp/1632594601_shops.xml.1633024799066.db. (part 2110/2120)
done -- 2110 parts merged (of 2120). total 0. time 0.016491174697875977 
process /tmp/1632596401_shops.xml.1633024804292.db. (part 2111/2120)
done -- 2111 parts merged (of 2120). total 0. time 0.023101806640625 
process /tmp/1632598201_shops.xml.1633024808945.db. (part 2112/2120)
done -- 2

'/home/ed/repo/github/pyTest/dataSet/haddan/midData/mid.db'